In [ ]:
##

In [1]:
# Import libraries
import pandas as pd
import json
import unicodedata

In [26]:
# Definitions of ids in drilldowns

sex_id = {
    "Mujeres": 1, 
    "Hombres": 2, 
    "Indeterminado": 3
}

URL_regiones = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTmpNxBMYOQpFzxVENltVuRX5Z1Gvg4Sf-Wb6KqiGsdRf6-zymY-nxEgmEC2U7nTy1kd7p8AQ8GR44A/pub?gid=2066209252&single=true&output=csv"
df_regiones = pd.read_csv(URL_regiones)
df_regiones["name"] = df_regiones["name"].str.normalize("NFKD").str.encode("ascii", errors="ignore").str.decode("utf-8").str.lower()
region_id = dict(zip(df_regiones["name"], df_regiones["id"]))

In [ ]:
tidy = []
URLS = [
    {
        "link": "http://www.deis.cl/wp-content/uploads/2016/08/3.-Defunciones-seg%C3%BAn-grupo-de-edad-regi%C3%B3n-gran-grupo-de-causas-y-sexo.-Chile-2014.xlsx",
        "year": 2014,
        "first_row": 7
    },
    {
        "link": "http://www.deis.cl/wp-content/uploads/2016/08/3.-Defunciones-seg%C3%BAn-grupo-de-edad-regi%C3%B3n-gran-grupo-de-causas-y-sexo.-Chile-2013.xlsx",
        "year": 2013,
        "first_row": 6
    }
]

def isNaN(num):
    return num != num

## Extract & Transform

In [23]:
for URL in URLS:
    data = pd.read_excel(URL["link"])
    # Read title/subtitle of Excel

    title = [ str(item).strip() for item in data.iloc[2] ]
    subtitle = [ str(item).strip() for item in data.iloc[3] ]
    
    cie10 = ""
    geo = ""

    # Read data
    for i in range(URL["first_row"], len(data)):
        frame = data.iloc[i]
        
        if str(frame[0]).strip() and isNaN(frame[1]) and not isNaN(frame[0]):
            geo = (frame[0]).strip()
        
        age_range = ""
        measure = ""

        sex = ""
        if frame[1] in ["Hombres", "Mujeres", "Indeterminado"]:
            sex = frame[1]
        
        if frame[2] in ["Hombres", "Mujeres", "Indeterminado"]:
            sex = frame[2]
            
        elif not isNaN(frame[1]) and not isNaN(frame[0]):
            cie10 = frame[0].strip()
        
        if cie10 != "" and isNaN(frame[0]) and frame[1] != "Resto de causas":
            for tup in list(zip(title, subtitle, frame))[4:][:-1]:
                if tup[1] != "nan":
                    age_range = tup[1]
                if tup[2] != "nan":
                    measure = tup[2]
                if tup[0] != "":
                    age_range_id = int((int(age_range.split(" ")[0]) / 5) + 1)
                    tidy.append({
                        "geo": region_id[unicodedata.normalize("NFKD", geo).encode("ascii", errors="ignore").decode("utf-8").lower()],
                        "sex": sex_id[sex],
                        "cie10": cie10,
                        "age_range": age_range_id,
                        "count": measure,
                        "year": URL["year"]
                    })

## Load

In [24]:
# Save data in CSV
pd.DataFrame(tidy).to_csv("data/mortality.csv", index=False)

In [ ]:
pd.DataFrame(tidy).head()